<a href="https://colab.research.google.com/github/daliaydom/Aprendizaje-Profundo/blob/main/Ejercicio2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Curso de aprendizaje profundo
## Redes densas
### Dalia Yvette Domínguez Jiménez

In [26]:
import numpy as np
import pandas as pd

#### Ejercicio 2: Retropropagación en red densa

Programa el algoritmo de retropropagación usando NumPy para una tarea de clasificación binaria presuponiendo una red densa con dos capas ocultas. Esta red tiene una función de activación logística en todas sus neuronas y se entrena minimizando la función de pérdida de entropía cruzada binaria. Describe las fórmulas y reglas de actualización de los pesos y sesgos de cada capa y entrena y evalúa la red en algún conjunto de datos.

La función de activación se define como:

$$
\sigma(z) = \frac{1}{1 + e^{-z}}
$$

In [27]:
def sigmoide(z):
    return 1 / (1 + np.exp(-z))

La función sigmoide tiene una derivada que está expresada en términos de la misma función, esto es, 

$$
\frac{\partial \sigma (z)}{\partial z} = \sigma(z) (1 - \sigma(z))
$$

In [28]:
def derivada_sigmoide(x):
    return np.multiply(sigmoide(x), (1.0 - sigmoide(x)))

La función de pérdida de entropía cruzada binaria se define como:

$$
ECB(\mathbf{y}, \mathbf{\hat{y}})  = -\sum_{i=1}^N \left[ y^{(i)} \log \hat{y}^{(i)} + (1 - y^{(i)}) \log (1 - \hat{y}^{(i)}) \right]
$$

In [29]:
def entropia_cruzada_binaria(y, p):
    p[p == 0] = np.nextafter(0., 1.)
    p[p == 1] = np.nextafter(1., 0.)
    return -(np.log(p[y == 1]).sum() + np.log(1 - p[y == 0]).sum())

Asimismo, calcularemos la exactitud para medir el rendimiento del modelo aprendido por la red neuronal densa:

$$
exactitud = \frac{correctos}{total}
$$

In [30]:
def exactitud(y, y_predicha):
    return (y == y_predicha).mean() * 100

Ahora, definimos la función que propaga hacia adelante una entrada $\mathbf{x}^{i}$. Como la red está compuesta de 2 capas ocultas, tenemos 3 matrices de pesos con sus correspondientes vectores de sesgos $\{\mathbf{W}^{\{1\}}, \mathbf{b}^{\{1\}}\}$, $\{\mathbf{W}^{\{2\}}, \mathbf{b}^{\{2\}}\}$ y $\{\mathbf{W}^{\{3\}}, \mathbf{b}^{\{3\}}\}$ y una capa o neurona de salida. Así, podemos llevar a cabo la propagación hacia adelante en esta red de la siguiente manera:


$$
	\begin{split}
				\mathbf{a}^{\{0\}} & =  \mathbf{x}^{(i)} \\
				\mathbf{z}^{\{1\}} & =  \mathbf{W}^{\{1\}} \cdot \mathbf{a}^{\{0\}} + \mathbf{b}^{\{1\}}= f_1( \mathbf{W}^{\{1\}}, \mathbf{b}^{\{1\}},\mathbf{a}^{\{0\}})\\
				\mathbf{a}^{\{1\}} & =  \sigma(\mathbf{z}^{\{1\}}) = f_2( \emptyset, \mathbf{z}^{\{1\}})\\
				\mathbf{z}^{\{2\}} & =  \mathbf{W}^{\{2\}} \cdot \mathbf{a}^{\{1\}}  + \mathbf{b}^{\{2\}}= f_3( \mathbf{W}^{\{2\}}, \mathbf{b}^{\{2\}},\mathbf{a}^{\{1\}})\\
				\mathbf{a}^{\{2\}} & =  \sigma(\mathbf{z}^{\{2\}}) = f_4( \emptyset, \mathbf{z}^{\{2\}},\mathbf{a}^{\{0\}})\\
				\mathbf{z}^{\{3\}} & =  \mathbf{W}^{\{3\}} \cdot \mathbf{a}^{\{2\}}  + \mathbf{b}^{\{3\}}= f_5( \mathbf{W}^{\{3\}}, \mathbf{b}^{\{3\}})\\
				\mathbf{a}^{\{3\}} & =  \sigma(\mathbf{z}^{\{2\}})=\hat{y}^{(i)} = f_6( \emptyset, \mathbf{z}^{\{3\}})
			\end{split}
      $$

In [31]:
def forward(x,W1,b1,W2,b2,W3,b3):
    
    z1 = np.dot(W1.T,x[:, np.newaxis])+b1
    a1 = sigmoide(z1)
    
    z2 = np.dot(W2.T,a1)+b2
    a2 = sigmoide(z2)
    
    z3 = np.dot(W3.T,a2)+b3
    y_hat = sigmoide(z3)
    
    return z1, a1, z2, a2, z3, y_hat

Finalmente, definimos la función para entrenar nuestra red neuronal usando gradiente descendente. Para calcular el gradiente de la función de pérdida respecto a los pesos y sesgos en cada capa empleamos el algoritmo de retropropagación.


In [32]:
def retropropagacion(X, y, alpha = 0.01, n_epocas = 100, n_ocultas_capa1 = 2, n_ocultas_capa2 = 4):
    n_ejemplos = X.shape[0]
    n_entradas = X.shape[1]
        
    # Inicialización de las matrices de pesos W y V
    W1 = np.sqrt(1.0 / n_entradas) * np.random.randn(n_entradas, n_ocultas_capa1)
    b1 = np.zeros((n_ocultas_capa1, 1))
    
    W2 = np.sqrt(1.0 / n_ocultas_capa1) * np.random.randn(n_ocultas_capa1, n_ocultas_capa2)
    b2 = np.zeros((n_ocultas_capa2, 1))
    
    W3 = np.sqrt(1.0 / n_ocultas_capa2) * np.random.randn(n_ocultas_capa2, 1)
    b3 = np.zeros((1, 1))
    
    perdidas = np.zeros((n_epocas))
    exactitudes = np.zeros((n_epocas))
    y_predicha = np.zeros((y.shape))
    for i in range(n_epocas):
        for j in range(n_ejemplos):
            
            z1, a1, z2, a2, z3, y_hat = forward(X[j],W1,b1,W2,b2,W3,b3)
            
            # cálculo de gradientes para W3 y b3 por retropropagación
            dz4 = (y_hat - y[j])
            dW3 = np.outer(a2, dz4)
            db3 = dz4
            
            # cálculo de gradientes para W2 y b2 por retropropagación/
            dz3 = np.dot(W3, dz4) * derivada_sigmoide(z2)
            dW2 = np.outer(a1, dz3)
            db2 = dz3
            
            # cálculo de gradientes para W1 y b1 por retropropagación
            dz2 = np.dot(W2, dz3) * derivada_sigmoide(z1)
            dW1 = np.outer(X[j], dz2)
            db1 = dz2

            ####################################
            # IMPORTANTE 
            # la actualización de los parámetros
            # debe hacerse de forma simultánea
            W3 = W3 - alpha * dW3
            b3 = b3 - alpha * db3
            
            W2 = W2 - alpha * dW2
            b2 = b2 - alpha * db2            
            W1 = W1 - alpha * dW1
            b1 = b1 - alpha * db1

            y_predicha[j] = y_hat
            
        # calcula la pérdida en la época
        perdidas[i] = entropia_cruzada_binaria(y, y_predicha)
        exactitudes[i] = exactitud(y, np.round(y_predicha))
        print('Epoch {0}: Pérdida = {1} Exactitud = {2}'.format(i, 
                                                              perdidas[i], 
                                                              exactitudes[i]))

    return W1, W2, W3, perdidas, exactitudes

## Entrenamieno de red con conjunto de datos

Vamos a utilizar el conjunto de datos de tumores de seno, el cual se utilizó el semestre pasado.

### Lectura de datos 

El conjunto de datos contiene 699 registros de tumores de seno, de los cuales 458 son benignos y 241 son malignos. En donde la clase es 2 para benigno y 4 para maligno.

In [33]:
URL = 'https://raw.githubusercontent.com/daliaydom/Aprendizaje-Profundo/main/breast-cancer-wisconsin.data'

! wget -nc {URL} -O {'breast-cancer-wisconsin.data'}


--2022-09-28 06:16:00--  https://raw.githubusercontent.com/daliaydom/Aprendizaje-Profundo/main/breast-cancer-wisconsin.data
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 19889 (19K) [text/plain]
Saving to: ‘breast-cancer-wisconsin.data’

breast-cancer-wisco 100%[===================>]  19.42K  --.-KB/s    in 0.002s  

2022-09-28 06:16:00 (8.46 MB/s) - ‘breast-cancer-wisconsin.data’ saved [19889/19889]



In [34]:
df = pd.read_csv('breast-cancer-wisconsin.data')
df.columns= ['Código de la muestra',
          'Grosor del tumor',
          'Uniformidad del tamño',
          'Uniformidad de la forma',
          'Adhesión marginal',
          'Tamaño',
          'Núcleos desnudos',
          'Cromatina blanda',
          'Nucléolos normales',
          'Mitosis de células',
          'Clase']
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 698 entries, 0 to 697
Data columns (total 11 columns):
 #   Column                   Non-Null Count  Dtype 
---  ------                   --------------  ----- 
 0   Código de la muestra     698 non-null    int64 
 1   Grosor del tumor         698 non-null    int64 
 2   Uniformidad del tamño    698 non-null    int64 
 3   Uniformidad de la forma  698 non-null    int64 
 4   Adhesión marginal        698 non-null    int64 
 5   Tamaño                   698 non-null    int64 
 6   Núcleos desnudos         698 non-null    object
 7   Cromatina blanda         698 non-null    int64 
 8   Nucléolos normales       698 non-null    int64 
 9   Mitosis de células       698 non-null    int64 
 10  Clase                    698 non-null    int64 
dtypes: int64(10), object(1)
memory usage: 60.1+ KB


Cambiamos las variables de clases por variables binarias 0 para benigno (antes 2) y 1 para maligno (antes 4).

In [35]:
df['Clase']=df['Clase'].replace(2,0)
df['Clase']=df['Clase'].replace(4,1)

Eliminamos las filas en donde la columna Núcleos desnudos tiene un ?

In [36]:
indexes = list(df.index[df['Núcleos desnudos']=='?'])
df_delete=df.drop(indexes)
df_delete = df_delete.astype({"Núcleos desnudos": 'int64'})
df_delete.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 682 entries, 0 to 697
Data columns (total 11 columns):
 #   Column                   Non-Null Count  Dtype
---  ------                   --------------  -----
 0   Código de la muestra     682 non-null    int64
 1   Grosor del tumor         682 non-null    int64
 2   Uniformidad del tamño    682 non-null    int64
 3   Uniformidad de la forma  682 non-null    int64
 4   Adhesión marginal        682 non-null    int64
 5   Tamaño                   682 non-null    int64
 6   Núcleos desnudos         682 non-null    int64
 7   Cromatina blanda         682 non-null    int64
 8   Nucléolos normales       682 non-null    int64
 9   Mitosis de células       682 non-null    int64
 10  Clase                    682 non-null    int64
dtypes: int64(11)
memory usage: 63.9 KB


In [37]:
df = df_delete
X=df[df.columns[1:-1]].to_numpy(dtype=float)
y=df[df.columns[-1]].to_numpy()[np.newaxis:].T

In [38]:
np.random.seed(0)
W1, W2, w3, perdidas, exactitudes = retropropagacion(X, 
                                                 y, 
                                                 alpha = 0.01,
                                                 n_epocas = 40)

Epoch 0: Pérdida = 444.7210752145709 Exactitud = 61.43695014662757
Epoch 1: Pérdida = 433.4297617670982 Exactitud = 64.95601173020528
Epoch 2: Pérdida = 425.3479135394575 Exactitud = 64.95601173020528
Epoch 3: Pérdida = 410.7222113989718 Exactitud = 64.95601173020528
Epoch 4: Pérdida = 388.0535938531217 Exactitud = 68.62170087976538
Epoch 5: Pérdida = 357.32347557488583 Exactitud = 72.72727272727273
Epoch 6: Pérdida = 321.4236734595304 Exactitud = 84.4574780058651
Epoch 7: Pérdida = 284.49437062606694 Exactitud = 91.49560117302052
Epoch 8: Pérdida = 249.89155214841279 Exactitud = 93.841642228739
Epoch 9: Pérdida = 219.9373563630284 Exactitud = 93.54838709677419
Epoch 10: Pérdida = 194.7260518814519 Exactitud = 93.54838709677419
Epoch 11: Pérdida = 174.17707396355848 Exactitud = 93.69501466275659
Epoch 12: Pérdida = 157.93974910355365 Exactitud = 94.42815249266863
Epoch 13: Pérdida = 144.75308449467516 Exactitud = 94.57478005865103
Epoch 14: Pérdida = 133.6352741580925 Exactitud = 95.01